<a href="https://colab.research.google.com/github/HughClaimsend/MA5851_HughMcMullan_A3/blob/main/MA5851_HughMcMullan_A3_Part1_Scraping_Drone_Data_with_Selenium_and_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing Selenium packages for crawling and scraping web pages
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
#Import other libraries
from bs4 import BeautifulSoup     # BeautifulSoup for parsing scraped pages
import re                         # Regex functions
import csv                        # Writing to csv file
import datetime                   # Generating timestamps

In [ ]:
# Define Selenium driver using Chrome libraries
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# function to fetch and scrape a website page
def fetch_and_scrape_website(url):
  # Fetch the web page [with Selenium]
  print("\nFETCHING ...", url)
  driver.get(url)       

  # Selenium hands the page source to Beautiful Soup
  soup = BeautifulSoup(driver.page_source, 'lxml')

  # Closes all browser windows and safely ends the session
  #driver.close()

  # return the soup object
  return(soup)

In [ ]:
def strip_thread(thread):
  rtn = thread
  if thread[-12:-7] == "post-":
    rtn = thread[:-12]
  return(rtn)

In [ ]:
# function to scrape a set of SEARCH RESULT pages using a page_no
def scrape_and_save_search_pages(dronesite_details_list, max_pages, max_items, output_filename):
  dronesite_name = dronesite_details_list[0]
  url_base = dronesite_details_list[1]
  search_identifier = dronesite_details_list[2]
  dronesite_group = dronesite_details_list[3]
  # print(dronesite_name)
  # print(dronesite_group)
  # print(url_base)
  # print(search_identifier)
  
  # syntax for mavic/phantom etc websites
  search_url_base = url_base + "/search/" + search_identifier + "/?q=casa&o=relevance"
  # print("Search thread:", search_url_base)
  # print("\n")

  for page in range(max_pages):    
    url_page = search_url_base + "&page=" + str(page + 1)   # incorporate search page no 
    soup = fetch_and_scrape_website(url_page)               # load the page into soup
    
    # Run through all the threads found by the search
    threads = soup.find_all('h3',{'class':'contentRow-title'})
    items_count = 0
    for thread in threads:
      href = thread.find("a")["href"]
      thread_name = thread.find("a").text

      # Print this block so scraping can be monitored
      print("\n=====================================================================")
      print("Search thread:", thread_name)
      print("=====================================================================")

      thread_url = url_base + strip_thread(href)

      scrape_and_save_thread_pages(dronesite_name, dronesite_group, thread_name, thread_url, max_items, output_filename)

      items_count += 1
      if items_count > max_items:
          break


In [ ]:
# function to scrape the FIRST THREAD page (even if there are multiple pages)
def scrape_and_save_thread_pages(dronesite_name, dronesite_group, thread_name, thread_url, max_items, output_filename):
  thread_no = thread_url[-7:]
  thread_no = thread_no[:-1]
  #print(thread_no, thread_name, thread_url)

  soup = fetch_and_scrape_website(thread_url)               # load the page into soup
    
  # Run through all the threads found by the search
  posts = soup.find_all('div',{'class':'message-main'})
  items_count = 0
  for post in posts:
    # Find and extract post data from the thread html
    post_datetime = post.find('time')["datetime"]
    post_article = post.find('article')
    post_text = post_article.find('div',{'class':'bbWrapper'})
    post_text = post_text.text
    # print("\n--------------------------------------------------------------------")
    # print("Posted at:", post_datetime)
    # print("--------------------------------------------------------------------")
    # print("Post text:", post_text)

    # Save the posts data
    list_to_save = [dronesite_name, dronesite_group, thread_name, thread_no, post_datetime, post_text]
    save_post_to_csv(output_filename, list_to_save)

    # Stop the loop if max no of post items is reached
    items_count += 1
    if items_count > max_items:
        break

In [ ]:
# Write the passed-in list of post details to a line in the .csv file
def save_post_to_csv(output_filename, list_input):
  try:
    with open(output_filename, "a" ) as fopen:    # Open the csv file
      csv_writer = csv.writer(fopen)
      csv_writer.writerow(list_input)
  except:
      return False


In [ ]:
# Set a list website targets for posts scraping
drone_website_targets = [
                         ["MavicPilots", "https://mavicpilots.com", "707065", "Private"],
                         ["PhantomPilots", "https://phantompilots.com", "213313", "Private"],
                         ["FPVDronePilots", "https://fpvdronepilots.com", "15671", "Private"],
                         ["CommercialDronePilots", "https://commercialdronepilots.com", "18113", "Commercial"],
                         ["MatricePilots", "https://matricepilots.com", "49374", "Commercial" ],
                         ["InspirePilots", "https://inspirepilots.com", "56174", "Commercial"]
]

# Set arbitrary limits to avoid too many calls to website(s)
max_page_calls = 10
max_items_per_page = 20

# Initialise dronesite_output file
datetime_now = datetime.datetime.now()
timestamp = datetime_now.strftime("%Y%m%d%H%M")
dronesite_output_filename = "DronePosts_" + timestamp + ".csv"
#print(dronesite_output_filename)

In [ ]:
# Run through the 'CASA' Search pages for each target website
for dronesite_details_list in drone_website_targets:
  scrape_and_save_search_pages(dronesite_details_list, max_page_calls, max_items_per_page, dronesite_output_filename)


Streaming output truncated to the last 5000 lines.
Search thread: Is it illegal to fly over main roads

FETCHING ... https://mavicpilots.com/threads/is-it-illegal-to-fly-over-main-roads.86652/post-981987

Search thread: Can authorities detect a Mavic mini?

FETCHING ... https://mavicpilots.com/threads/can-authorities-detect-a-mavic-mini.110780/

Search thread: Can I fly at night?

FETCHING ... https://mavicpilots.com/threads/can-i-fly-at-night.92774/

Search thread: New Member

FETCHING ... https://mavicpilots.com/threads/new-member.91501/

Search thread: New flyer in Melbourne, AUS.

FETCHING ... https://mavicpilots.com/threads/new-flyer-in-melbourne-aus.88530/

Search thread: No Fly Zone updates

FETCHING ... https://mavicpilots.com/threads/no-fly-zone-updates.92304/

FETCHING ... https://mavicpilots.com/search/707065/?q=casa&o=relevance&page=10

Search thread: Auto land on low battery

FETCHING ... https://mavicpilots.com/threads/auto-land-on-low-battery.92186/

Search thread: Mavic